<a href="https://colab.research.google.com/github/brianriceca/titanic/blob/main/titanic_tf_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic Challenge

In [ ]:
import numpy as np
!pip install plotly
import plotly.express as px

In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')

Now let's explore some scatters.

In [ ]:
train.head()

,Id,Outcome,Pclass,Name,Sex,Age,Sibsp,Parch,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,3,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [ ]:
survivors = train[train.Outcome == 1]
survivors.head()
sinkers = train[train.Outcome == 0]
sinkers.head()

,Id,Outcome,Pclass,Name,Sex,Age,Sibsp,Parch,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,6,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,7,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,8,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750


In [ ]:

fig1 = px.pie(survivors, title=f'Class of {survivors.shape[0]} survivors', values="Pclass",names="Pclass")
fig1.show()
fig2 = px.pie(sinkers, title=f'Class of {sinkers.shape[0]} sinkers', values="Pclass",names="Pclass")
fig2.show()

In [ ]:
onehot = pd.get_dummies(train, columns=['Pclass','Sex'])
onehot.head()

,Id,Outcome,Name,Age,Sibsp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,0,Mr. Owen Harris Braund,22.0,1,0,7.2500,0,0,1,0,1
1,2,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,38.0,1,0,71.2833,1,0,0,1,0
2,3,1,Miss. Laina Heikkinen,26.0,0,0,7.9250,0,0,1,1,0
3,4,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,35.0,1,0,53.1000,1,0,0,1,0
4,5,0,Mr. William Henry Allen,35.0,0,0,8.0500,0,0,1,0,1


In [ ]:
bins = pd.IntervalIndex.from_tuples([(0, 2), (2, 9), (9, 18), (18,999)])
ages = train['Age']
agebrackets = pd.cut(ages, bins=bins, labels=['Infant','Child','Teen','Adult','Elderly'] )
agebrackets.head()

0    (18, 999]
1    (18, 999]
2    (18, 999]
3    (18, 999]
4    (18, 999]
Name: Age, dtype: category
Categories (4, interval[int64, right]): [(0, 2] < (2, 9] < (9, 18] < (18, 999]]